###Fuente de la data

Los micro-datos gestionados en este notebook son tomados de www.datos.gov.co y corresponden a datos cuya fuente primaria es el Ministerio de Defensa Nacional de Colombia

In [1]:
import pandas as pd

###Archivo 9zck-qfvc.csv

Contiene información sobre los delitos contra los recursos naturales y el medio ambiente descritos en este documento son todos los que están comprendidos en el Título XI “De los delitos contra los recursos naturales y el medio ambiente” del Código Penal Colombiano, artículos 328 al 339. Los delitos que se muestran a continuación corresponden a la modificación que la Ley 2111 de 2021 (29 de julio) dispuso para este Título. Se mide en casos.sobre homicidios. Toda muerte (civiles, miembros de la Fuerza Pública y personas pertenecientes a grupos criminales), que fuere causada por otra persona por cualquier tipo de elemento. Se mide en víctimas.

In [2]:
df = pd.read_csv("https://www.datos.gov.co/resource/9zck-qfvc.csv?$limit=500000", dtype={'cod_muni': str})
df.head()

,fecha_hecho,cod_depto,departamento,cod_muni,municipio,descripcion_conducta,zona,cantidad
0,2024-07-31T00:00:00.000,5,ANTIOQUIA,05088,BELLO,ARTICULO 328. APROVECHAMIENTO ILICITO DE LOS R...,RURAL,1
1,2024-07-31T00:00:00.000,13,BOLIVAR,13001,CARTAGENA DE INDIAS,ARTICULO 334. CONTAMINACION AMBIENTAL,URBANA,1
2,2024-07-31T00:00:00.000,23,CORDOBA,23189,CIENAGA DE ORO,ARTICULO 328. APROVECHAMIENTO ILICITO DE LOS R...,RURAL,1
3,2024-07-31T00:00:00.000,23,CORDOBA,23300,COTORRA,ARTICULO 328. APROVECHAMIENTO ILICITO DE LOS R...,URBANA,1
4,2024-07-31T00:00:00.000,86,PUTUMAYO,86568,PUERTO ASIS,ARTICULO 328. APROVECHAMIENTO ILICITO DE LOS R...,RURAL,1


##Revisión y limpieza para integrarlo a la base de datos

### - Resumen de la estructura del dataset

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71362 entries, 0 to 71361
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   fecha_hecho           71362 non-null  object
 1   cod_depto             71362 non-null  int64 
 2   departamento          71362 non-null  object
 3   cod_muni              71362 non-null  object
 4   municipio             71362 non-null  object
 5   descripcion_conducta  71362 non-null  object
 6   zona                  71362 non-null  object
 7   cantidad              71362 non-null  int64 
dtypes: int64(2), object(6)
memory usage: 4.4+ MB


### Eliminación de columnas irrelevantes para el proyecto

In [4]:
relevant_cols = ['cod_muni', 'departamento', 'municipio', 'descripcion_conducta', 'zona', 'fecha_hecho', 'cantidad']
environmental_crimes_MinDefensa = df[relevant_cols]
environmental_crimes_MinDefensa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71362 entries, 0 to 71361
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   cod_muni              71362 non-null  object
 1   departamento          71362 non-null  object
 2   municipio             71362 non-null  object
 3   descripcion_conducta  71362 non-null  object
 4   zona                  71362 non-null  object
 5   fecha_hecho           71362 non-null  object
 6   cantidad              71362 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 3.8+ MB


### Convertir columna fecha_hecho a tipo date

In [5]:
# Hacer una copia explícita del DataFrame
environmental_crimes_MinDefensa_ = environmental_crimes_MinDefensa.copy()

# Convertir la columna 'fecha_hecho' a tipo datetime (sin especificar formato)
environmental_crimes_MinDefensa_['fecha_hecho'] = pd.to_datetime(environmental_crimes_MinDefensa_['fecha_hecho'], errors='coerce')

# Extraer solo el año
environmental_crimes_MinDefensa_['fecha_hecho'] = environmental_crimes_MinDefensa_['fecha_hecho'].dt.year

In [6]:
environmental_crimes_MinDefensa_['fecha_hecho'].unique()

array([2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014,
       2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003],
      dtype=int32)

### Verificar valores nulos

In [ ]:
environmental_crimes_MinDefensa_.isnull().sum()

,0
cod_muni,0
departamento,0
municipio,0
descripcion_conducta,0
zona,0
fecha_hecho,0
cantidad,0


### Estandarización de categorizaciones

La estandarización de categorizaciones es el proceso de uniformizar y normalizar los valores de las categorías en un conjunto de datos para asegurar la consistencia y evitar discrepancias. Esto es crucial para la calidad y precisión de los análisis

In [ ]:
# Imprimir categorías únicas para columnas de tipo object
categorical_col = ['departamento', 'municipio', 'descripcion_conducta', 'zona']
for column in categorical_col:
    print(f"Categorías en la columna '{column}':")
    print(environmental_crimes_MinDefensa_[column].unique())
    print()


Categorías en la columna 'departamento':
['ANTIOQUIA' 'BOLIVAR' 'CORDOBA' 'PUTUMAYO' 'SANTANDER' 'MAGDALENA'
 'TOLIMA' 'BOGOTA D.C.' 'NARIÑO' 'SUCRE' 'CAQUETA' 'CASANARE'
 'CUNDINAMARCA' 'HUILA' 'META' 'BOYACA' 'CALDAS' 'VALLE DEL CAUCA'
 'RISARALDA' 'CESAR' 'CHOCO' 'LA GUAJIRA' 'ARAUCA' 'CAUCA' 'ATLANTICO'
 'NORTE DE SANTANDER' 'GUAINIA' 'SAN ANDRES ISLAS' 'VICHADA' 'QUINDIO'
 'VAUPES' 'GUAVIARE' 'AMAZONAS']

Categorías en la columna 'municipio':
['BELLO' 'CARTAGENA DE INDIAS' 'CIENAGA DE ORO' ... 'LA CAPILLA'
 'SAN JOSE DEL PALMAR' 'BITUIMA']

Categorías en la columna 'descripcion_conducta':
['ARTICULO 328. APROVECHAMIENTO ILICITO DE LOS RECURSOS NATURALES RENOVABLES'
 'ARTICULO 334. CONTAMINACION AMBIENTAL'
 'ARTICULO 332. EXPLOTACION ILICITA DE YACIMIENTO MINERO Y OTROS MATERIALES'
 'ARTICULO 336. INVASION DE AREAS DE ESPECIAL IMPORTANCIA ECOLOGICA'
 'ARTICULO 333. DAÑOS EN LOS RECURSOS NATURALES Y ECOCIDIO'
 'ARTICULO 330. DEFORESTACION'
 'ARTICULO 334A. CONTAMINACION AMBIENTAL PO

- Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños

In [ ]:
import unicodedata

def remove_accents_and_special_chars(input_str):
    # Normalizar la cadena a NFKD
    nfkd_form = unicodedata.normalize('NFKD', input_str)

    # Eliminar acentos
    no_accents = ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

    # Definir caracteres no deseados
    unwanted_chars = [',', ';', '!', '?', '#', '$', '%']

    # Eliminar caracteres no deseados
    cleaned_str = ''.join([c for c in no_accents if c not in unwanted_chars])

    # Remover espacios en blanco al principio y al final, y convertir a mayúsculas
    result = cleaned_str.strip().upper()

    return result

In [ ]:
# Aplicar la función a todas las columnas categóricas
for col in categorical_col:
    environmental_crimes_MinDefensa_[col] = environmental_crimes_MinDefensa_[col].apply(remove_accents_and_special_chars)

- Codificación de algunas variables categóricas

In [ ]:
# Reemplazar valores en la columna 'zona'
environmental_crimes_MinDefensa_.loc[:, 'zona'] = environmental_crimes_MinDefensa_['zona'].replace({'URBANA': 1, 'RURAL': 0})

### Verificar que los valores en "cod_muni" coincidan con los códigos reales de municipios

Los códigos reales de los municipios de Colombia, están almacenados en la base de datos PostgreSQL del proyecto, en la tabla municipalities dentro del campo dept_mpio_code, junto con la informacion necesaria para georeferenciar todos los municipios y departamentos de Colombia. Este campo guarda el código del municipio en un formato string de exactamente 5 caracteres.

Para hacer esta verificación, previamente exportamos desde la base de datos PostgreSQL un DataFrame con los siguientes campos: dept_name, mpio_name y dept_mpio_code, los cuales contienen la información de los departamentos y municipios oficiales, junto con sus respectivos códigos. Este DataFrame se carga en la siguiente celda y se utiliza para comparar con la columna "codigo_mpio"

In [ ]:
dept_mpios_codes = pd.read_csv("/content/drive/MyDrive/analytics_data_proyect/deptos_mupios.csv", index_col=0, dtype={'dept_mpio_code': str})
print(dept_mpios_codes.info())
dept_mpios_codes.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 0 to 1120
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dept_mpio_code  1121 non-null   object
 1   dept_name       1121 non-null   object
 2   mupio_name      1121 non-null   object
dtypes: object(3)
memory usage: 35.0+ KB
None


,dept_mpio_code,dept_name,mupio_name
0,97001,VAUPES,MITU
1,97161,VAUPES,CARURU
2,97511,VAUPES,PACOA
3,97666,VAUPES,TARAIRA
4,97777,VAUPES,PAPUNAHUA


 -  Verificar la consistencia de la columna "cod_muni" en el df environmental_crimes_MinDefensa_

In [ ]:
# Asegurarnos de que todos los valores en 'codigo_m' sean strings
environmental_crimes_MinDefensa_['cod_muni'] = environmental_crimes_MinDefensa_['cod_muni'].astype(str)

# Calcular la longitud de cada valor en la columna
longitudes = environmental_crimes_MinDefensa_['cod_muni'].apply(len)

# Verificar si todas las longitudes son iguales
longitudes.nunique() == 1

True

In [ ]:
# Mostrar longitudes únicas (opcional)
print(f"Longitudes únicas: {longitudes.unique()}")

Longitudes únicas: [5]


In [ ]:
# Contar registros por longitud
long_df = longitudes.value_counts().reset_index()
long_df.rename(columns={'codigo_dane': 'no_dígitos_codigo_dane'}, inplace=True)
long_df['percentage'] = (long_df['count'] / len(longitudes))
long_df.head()

,cod_muni,count,percentage
0,5,71362,1.0


In [ ]:
# Mostrar una muestra de registros para cada longitud
for longitud in longitudes.value_counts().index:
    print(f"Muestra de registros con longitud {longitud}:")
    muestra = environmental_crimes_MinDefensa_[longitudes == longitud].head(5)  # Muestra de los primeros 5 registros
    print(muestra[['cod_muni']])
    print()

Muestra de registros con longitud 5:
  cod_muni
0    05088
1    13001
2    23189
3    23300
4    86568



Nota: el 100%  de los codigos que identifican a los municipios en el df environmental_crimes_MinDefensa corresponden a string de 5 caracteres, igual a los almacenados en la base de datos del proyecto

- Comparar que los nombres de los departamentos en cada dataframe esten escritos correctamente

Se aplica la función para racionalizar columnas categoricas (Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños) al df 'dept_mpiios_codes'


In [ ]:
# Aplicar función a columnas 'dept_name', 'mupio_name'
for col in dept_mpios_codes[['dept_name', 'mupio_name']]:
    dept_mpios_codes[col] = dept_mpios_codes[col].apply(remove_accents_and_special_chars)

In [ ]:
# Función para comparar listas y mostrar diferencias
def compare_lists(df1_col, df2_col, label1, label2):
    # Extraer listas únicas y normalizar
    list1 = set(df1_col.str.strip().str.upper().unique())
    list2 = set(df2_col.str.strip().str.upper().unique())

    # Encontrar diferencias
    only_in_list1 = list1 - list2
    only_in_list2 = list2 - list1

    # Imprimir resultados
    print(f"{label1} que no están en {label2}:")
    print(only_in_list1)
    # print(f"{label2} que no están en {label1}:")
    # print(only_in_list2)
    # print()


  - Comparar nombres de departamentos en los dataframes

In [ ]:
# Comparar listas de departamento
compare_lists(environmental_crimes_MinDefensa_['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en environmental_crimes_MinDefensa_", "Departamentos dept_mpios_codes")

Departamentos en environmental_crimes_MinDefensa_ que no están en Departamentos dept_mpios_codes:
{'SAN ANDRES ISLAS'}


  - Solucionar discrepancias en nombres de departamento

In [ ]:
# Diccionario de mapeo basado en los resultados de la comparación departamentos
depto_mapping = {'SAN ANDRES ISLAS': 'ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA'}

# Reemplazar los nombres incorrectos  el dataframe environmental_crimes_MinDefensa_
environmental_crimes_MinDefensa_['departamento'] = environmental_crimes_MinDefensa_['departamento'].replace(depto_mapping)

 - Confirmar que discrepancia se solucionó

In [ ]:
# Comparar listas de departamento
compare_lists(environmental_crimes_MinDefensa_['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en environmental_crimes_MinDefensa_", "Departamentos dept_mpios_codes")

Departamentos en environmental_crimes_MinDefensa_ que no están en Departamentos dept_mpios_codes:
set()


- Verificar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales

In [ ]:
# Comparar listas de códigos
compare_lists(environmental_crimes_MinDefensa_['cod_muni'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en environmental_crimes_MinDefensa_", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en environmental_crimes_MinDefensa_ que no están en Códigos de municipios dept_mpios_codes:
set()


### Procesamiento final como preparación para integrarlo a la bd de datos del proyecto

In [ ]:
# Eliminar columnas innecesarias
columns_to_drop = ['departamento', 'municipio']
final_environmental_crimes_MinDefensa = environmental_crimes_MinDefensa_.drop(columns=columns_to_drop)

In [ ]:
# Adicionar columna para trazabilidad de la fuente
final_environmental_crimes_MinDefensa['source_id'] = 22

In [ ]:
final_environmental_crimes_MinDefensa.columns

Index(['cod_muni', 'descripcion_conducta', 'zona', 'fecha_hecho', 'cantidad',
       'source_id'],
      dtype='object')

In [ ]:
# Ajustar nombre de columnas

# Definir el diccionario de traducción
translation_map = {
    'cod_muni': 'dane_code',
    'fecha_hecho': 'year_of_incident',
    'sexo': 'sex',
    'zona': 'area',
    'cantidad': 'amount',
    'source_id': 'source_id'
}

# Renombrar las columnas
final_environmental_crimes_MinDefensa.rename(columns=translation_map, inplace=True)

In [ ]:
#Estructura final del dataset a integrar a la base de datos
final_environmental_crimes_MinDefensa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71362 entries, 0 to 71361
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   dane_code             71362 non-null  object
 1   descripcion_conducta  71362 non-null  object
 2   area                  71362 non-null  object
 3   year_of_incident      71362 non-null  int32 
 4   amount                71362 non-null  int64 
 5   source_id             71362 non-null  int64 
dtypes: int32(1), int64(2), object(3)
memory usage: 3.0+ MB


## Salvar en archivo csv en el drive

In [ ]:
# Guardar en archivos CSV en el drive
final_environmental_crimes_MinDefensa.to_csv('/content/drive/MyDrive/analytics_data_proyect/initial_transformation/environmental_crimes_MinDefensa.csv', index=False)